# SUMÁRIO

- [1. INFOS API DATAJUD](#1-infos-api-datajud)

- [EXEMPLOS DE CONSULTA INDIVIDUAL](#testes-consulta-individual)

### 1. INFOS API DATAJUD

#### ENDPOINT

- https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search

#### AVALIAÇÃO DE LOTES COM PROBLEMA

In [ ]:
import streamlit as st
import pandas as pd
from time import time, sleep
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
import json
import os
from datetime import datetime

def consultar_processo(numero, url, headers, max_retries=3, delay=1.0):
    payload = json.dumps({"query": {"match": {"numeroProcesso": numero}}})
    for tentativa in range(1, max_retries + 1):
        try:
            response = requests.post(url, headers=headers, data=payload, timeout=20)
            response.raise_for_status()
            data = response.json()
            hits = data.get("hits", {}).get("hits", [])
            registros = []
            for hit in hits:
                registro = hit["_source"]
                registro["_id"] = hit.get("_id")
                registros.append(registro)
            sleep(delay)
            return (numero, registros, None)
        except Exception as e:
            if tentativa < max_retries:
                sleep(delay)
            else:
                return (numero, None, f"Tentativas esgotadas ({tentativa}): {e}")

def inspecionar_lote(lista_numeros, url, headers, lote_idx=0, n_threads=5):
    from pprint import pprint

    print(f"🔍 Inspecionando lote {lote_idx} com {len(lista_numeros)} processos...")
    resultados_brutos = []

    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        futures = {executor.submit(consultar_processo, numero, url, headers): numero for numero in lista_numeros}
        for future in as_completed(futures):
            numero = futures[future]
            try:
                _, registros, erro = future.result()
                if erro:
                    print(f"Erro no processo {numero}: {erro}")
                elif registros:
                    print(f"\n📄 Processo: {numero}")
                    for r in registros:
                        print("🔸 Registro bruto:")
                        pprint(r)
                        print("-" * 60)
                    resultados_brutos.extend(registros)
                else:
                    print(f"Processo {numero} não retornou dados.")
            except Exception as e:
                print(f"Erro inesperado no processo {numero}: {e}")

    print(f"\n✅ Total de registros brutos: {len(resultados_brutos)}")
    return resultados_brutos



def extrair_nomes_processo(lista_processos):
    resultados = []
    for processo in lista_processos:
        item = {}
        item['numeroProcesso'] = processo.get('numeroProcesso', None)

        # Extrair nome da classe
        classe = processo.get('classe', {})
        item['classe_nome'] = classe.get('nome') if isinstance(classe, dict) else None

        # Extrair nome do sistema
        sistema = processo.get('sistema', {})
        item['sistema_nome'] = sistema.get('nome') if isinstance(sistema, dict) else None

        # Extrair nome do formato
        formato = processo.get('formato', {})
        item['formato_nome'] = formato.get('nome') if isinstance(formato, dict) else None

        # Extrair nome do órgão julgador
        orgao = processo.get('orgaoJulgador', {})
        item['orgaoJulgador_nome'] = orgao.get('nome') if isinstance(orgao, dict) else None

        # Extrair lista de nomes dos assuntos
        assuntos = processo.get('assuntos', [])
        if isinstance(assuntos, list):
            item['assuntos_nome'] = [a.get('nome') for a in assuntos if isinstance(a, dict)]
        else:
            item['assuntos_nome'] = []

        # Extrair lista de nomes dos movimentos
        movimentos = processo.get('movimentos', [])
        if isinstance(movimentos, list):
            item['movimentos_nome'] = [m.get('nome') for m in movimentos if isinstance(m, dict)]
        else:
            item['movimentos_nome'] = []

        resultados.append(item)

    return resultados

def verificar_nomes_invalidos(lista_resultados):
    invalidos = []
    for i, item in enumerate(lista_resultados):
        # considera inválido se todos os campos de nome forem vazios
        campos = ['classe_nome', 'sistema_nome', 'formato_nome', 'orgaoJulgador_nome']
        if all(not item.get(c) for c in campos):
            invalidos.append((i, item['numeroProcesso']))
    return invalidos



In [ ]:
# Carregue os processos do lote 7366 (ou outro qualquer)
df_proc = pd.read_parquet("app10/dados-processos/tjsp_processos.parquet")
# lista_total = df_proc["Processo"].tolist()
# lote_idx = 7366
# batch_size = 100

# # Pegue os processos daquele lote
# inicio = (lote_idx - 1) * batch_size
# fim = inicio + batch_size
# lote_numeros = lista_total[inicio:fim]

# # Use a função de inspeção
# url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
# headers = {
#     'Authorization': 'ApiKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==',
#     'Content-Type': 'application/json'
# }
# inspecionar_lote(lote_numeros, url, headers, lote_idx)


In [ ]:
lista_total = df_proc["Processo"].tolist()
lote_idx = 7366
batch_size = 100

# Pegue os processos daquele lote
inicio = (lote_idx - 1) * batch_size
fim = inicio + batch_size
lote_numeros = lista_total[inicio:fim]

# Use a função de inspeção
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
headers = {
    'Authorization': 'ApiKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==',
    'Content-Type': 'application/json'
}

teste = inspecionar_lote(lote_numeros, url, headers, lote_idx)

In [ ]:
lote_idx = 1
batch_size = 100

# Pegue os processos daquele lote
inicio = (lote_idx - 1) * batch_size
fim = inicio + batch_size
lote_numeros = lista_total[inicio:fim]

teste_lote1 = inspecionar_lote(lote_numeros, url, headers, lote_idx)

In [ ]:
resultado_lote1 = extrair_nomes_processo(teste_lote1)

#### PROCESSOS NOVOS

In [ ]:
import pandas as pd

novos = pd.read_csv('./app7/dados-agrupados/TJSP_CN.csv', sep=";")

In [ ]:
import pandas as pd

chunks = []

for chunk in pd.read_csv("app7/dados-agrupados/TJSP_CN.csv", sep=";", chunksize=500_000, low_memory=False):
    chunks.append(chunk)

novos = pd.concat(chunks, ignore_index=True)

print(novos.shape)

In [ ]:
novos.head(3)

In [ ]:
proc_novos = novos[["Processo"]].copy()

proc_novos["Processo"] = proc_novos["Processo"].str.replace(r"[-.]", "", regex=True)
proc_novos = proc_novos[~proc_novos["Processo"].str.contains("sigiloso", case=False, na=False)]

lista_novos = proc_novos["Processo"].dropna().drop_duplicates().tolist()


In [ ]:
import requests
import json
import pandas as pd
import concurrent.futures
from time import sleep
from tqdm import tqdm

API_KEY = "cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
headers = {
    'Authorization': f'ApiKey {API_KEY}',
    'Content-Type': 'application/json'
}

payload = json.dumps({
    "query": {
        "match": {
            "numeroProcesso": "10008232820228260408"
            }
            }
        })

response = requests.post(url, headers=headers, data=payload, timeout=30)

data = response.json()


In [ ]:
data

In [ ]:
hits = data.get("hits", {}).get("hits", [])

In [ ]:
hits[0]['_source']

In [ ]:
import requests
import json
import pandas as pd
import concurrent.futures
from time import sleep
from tqdm import tqdm

API_KEY = "cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
headers = {
    'Authorization': f'ApiKey {API_KEY}',
    'Content-Type': 'application/json'
}

def consultar_processo(numero):
    payload = json.dumps({
        "query": {
            "match": {
                "numeroProcesso": numero
            }
        }
    })

    try:
        response = requests.post(url, headers=headers, data=payload, timeout=30)
        response.raise_for_status()
        data = response.json()

        resultados = []
        for hit in data.get("hits", {}).get("hits", []):
            registro = hit["_source"]
            registro["_id"] = hit.get("_id")
            resultados.append(registro)

        return resultados

    except Exception as e:
        print(f"❌ Erro no processo {numero}: {e}")
        return []

def processar_em_lotes(lista, max_threads=5):
    todos_resultados = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(consultar_processo, numero): numero for numero in lista}

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            resultado = future.result()
            todos_resultados.extend(resultado)
            sleep(0.1)  

    return todos_resultados


In [ ]:
import pandas as pd

parquet = pd.read_parquet('./app10/dados-processos/tjsp_processos.parquet')

#### TESTES CONSULTA INDIVIDUAL

[VOLTAR AO SUMÁRIO](#sumário)

In [ ]:
import requests
import json
import pandas as pd
import concurrent.futures
from time import sleep
from tqdm import tqdm

API_KEY = "cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
headers = {
    'Authorization': f'ApiKey {API_KEY}',
    'Content-Type': 'application/json'
}

def consultar_processo(numero):
    payload = json.dumps({
        "query": {
            "match": {
                "numeroProcesso": numero
            }
        }
    })

    try:
        response = requests.post(url, headers=headers, data=payload, timeout=30)
        response.raise_for_status()
        data = response.json()

        resultados = []
        for hit in data.get("hits", {}).get("hits", []):
            registro = hit["_source"]
            registro["_id"] = hit.get("_id")
            resultados.append(registro)

        return resultados

    except Exception as e:
        print(f"❌ Erro no processo {numero}: {e}")
        return []

def processar_em_lotes(lista, max_threads=5):
    todos_resultados = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(consultar_processo, numero): numero for numero in lista}

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            resultado = future.result()
            todos_resultados.extend(resultado)
            sleep(0.1)  

    return todos_resultados


In [ ]:
import pandas as pd

df = pd.DataFrame(consultar_processo('10203815120248260008'))[['dataHoraUltimaAtualizacao']]


In [ ]:
df

In [ ]:

df["safra"] = pd.to_datetime(df["dataHoraUltimaAtualizacao"]).dt.strftime("%Y%m%d")

df

In [ ]:
resultado = consultar_processo('10019441420248260411')
pd.DataFrame(resultado)

In [ ]:
consulta = consultar_processo("07809163020078260100")

pd.DataFrame(consulta)

#### AMOSTRA PARA TESTE NO STREAMLIT

In [ ]:
import pandas as pd
data_csv = pd.read_csv('./app7/dados-agrupados/TJSP_CPL_15anos.csv', sep=';')

data_csv.shape

In [ ]:
data_csv.to_parquet('./app7/dados-agrupados/parquet_teste.parquet')

#### RODAR STREAMLIT E GERAR O ARQUIVO .parquet

In [ ]:
import pandas as pd

teste_api = pd.read_parquet('./app7/resultados_api_teste/resultados_api_teste.parquet')

In [ ]:
teste_api.head(3)

In [ ]:
n_encontrado = consultar_processo("10017779420248260411")
n_encontrado

In [ ]:
n_encontrado2 = consultar_processo("15019178020248260408")

n_encontrado2

In [ ]:
pd.DataFrame(consultar_processo("15020806020248260408"))

In [ ]:
import pandas as pd
import pprint

df = pd.read_parquet('./app10/resultado-api/lotes/lote_001.parquet')



#### PROCESSOS PENDENTES

In [ ]:
proc_pendentes = pd.read_csv('./app7/dados-agrupados/TJSP_CPL.csv', sep=';')

#### PROCESSOS +15 ANOS PENDENTES

In [ ]:
proc_pendentes_15 = pd.read_csv('./app7/dados-agrupados/TJSP_CPL_15anos.csv', sep=';')

#### PROCESSOS JULGADOS

In [ ]:
proc_julgados = pd.read_csv('./app7/dados-agrupados/TJSP_Sent.csv', sep=';')

#### PROCESSOS BAIXADOS

In [ ]:
proc_baixados = pd.read_csv('./app7/dados-agrupados/TJSP_TBaix.csv', sep=';')

#### DADOS CORREGEDORIA

In [ ]:
import pandas as pd
# corregedoria = pd.read_csv('./app7/dados-agrupados/TJSP_tbl_correg.csv', sep=';')

data_csv = pd.read_csv('./app7/dados-agrupados/TJSP_CPL_15anos.csv', sep=';')


In [ ]:
import requests
import json
import pandas as pd
from time import sleep

url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"

# Headers fixos
headers = {
    'Authorization': 'ApiKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==',
    'Content-Type': 'application/json'
}

todos_resultados = []

for numero in lista_novos[:100]:
    payload = json.dumps({
        "query": {
            "match": {
                "numeroProcesso": numero
            }
        }
    })

    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()

        data = response.json()

        hits = data.get("hits", {}).get("hits", [])

        for hit in hits:
            registro = hit["_source"]
            registro["_id"] = hit.get("_id")  # adiciona id opcionalmente
            registro["numeroConsultado"] = numero  # adiciona o número original consultado
            todos_resultados.append(registro)

        print(f"✅ Sucesso: {numero} | Documentos encontrados: {len(hits)}")

    except Exception as e:
        print(f"❌ Erro no processo {numero}: {e}")
        continue

    sleep(1)  # intervalo para evitar sobrecarga na API

df_final = pd.DataFrame(todos_resultados)


In [ ]:
resultados = processar_em_lotes(lista_novos[:100], max_threads=5)

df_final = pd.DataFrame(resultados)
print(f"\n✅ Consulta finalizada. Total de registros: {len(df_final)}")

# df_final.to_parquet("datajud_resultados.parquet", index=False)

In [ ]:
from tqdm import tqdm
import requests
import json
import pandas as pd
from time import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def consultar_processo(numero, url, headers):
    payload = json.dumps({
        "query": {
            "match": {
                "numeroProcesso": numero
            }
        }
    })

    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        data = response.json()

        hits = data.get("hits", {}).get("hits", [])

        registros = []
        for hit in hits:
            registro = hit["_source"]
            registro["_id"] = hit.get("_id")
            registro["numeroConsultado"] = numero
            registros.append(registro)

        return (numero, registros, None)

    except Exception as e:
        return (numero, None, str(e))

def consultar_processos_paralelo(lista_novos, max_workers=5):
    url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
    headers = {
        'Authorization': 'ApiKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==',
        'Content-Type': 'application/json'
    }

    todos_resultados = []
    start_time = time()

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(consultar_processo, numero, url, headers): numero for numero in lista_novos}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Consultando processos"):
            numero = futures[future]
            try:
                num, registros, erro = future.result()
                if erro:
                    tqdm.write(f"❌ Erro no processo {num}: {erro}")
                else:
                    tqdm.write(f"✅ Sucesso: {num} | Documentos encontrados: {len(registros)}")
                    todos_resultados.extend(registros)
            except Exception as e:
                tqdm.write(f"❌ Erro inesperado no processo {numero}: {e}")

    end_time = time()
    tqdm.write(f"\n✅ Consulta realizada. Tempo total de processamento: {end_time - start_time:.2f} segundos.")

    df_final = pd.DataFrame(todos_resultados)
    return df_final
